In [2]:
import sqlite3
import pandas as pd



In [3]:
# Create/connect to SQLite database
conn = sqlite3.connect("country_club.db")  # this will create the DB file
cursor = conn.cursor()

In [12]:
sql_file = r"C:\Users\yang\Downloads\country_club.sql"
with open(sql_file, "r") as f:
    sql_script = f.read()
cursor.executescript(sql_script)
  

In [13]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('Bookings',), ('Facilities',), ('Members',)]


In [15]:
import pandas as pd
df = pd.read_sql_query("SELECT * FROM Bookings;", conn)

# Show first 5 rows
df.head()

,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


In [39]:
import sqlite3
import pandas as pd

# Connect to  database
conn = sqlite3.connect("country_club.db")


query = """
SELECT 
    f.name AS facility_name,
    SUM(
        CASE
            WHEN m.memid = 0 THEN f.guestcost * b.slots
            ELSE f.membercost * b.slots
        END
    ) AS total_revenue
FROM Bookings b
JOIN Members m ON m.memid = b.memid
JOIN Facilities f ON f.facid = b.facid
GROUP BY f.name
HAVING total_revenue < 1000
ORDER BY total_revenue ASC;
"""

# Load into pandas DataFrame
df_revenue = pd.read_sql_query(query, conn)

# Display the result
df_revenue

,facility_name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


In [40]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("country_club.db")


query = """
SELECT concat(m.surname,', ',m.firstname) as member_name,
concat(rm.surname,', ',rm.firstname) as recommend_name
FROM  Members m 
INNER JOIN Members rm ON m.recommendedby = rm.memid
ORDER BY m.surname, m.firstname ,rm.surname, rm.firstname

"""

# Load into pandas DataFrame
df_member = pd.read_sql_query(query, conn)

# Display the result
df_member



,member_name,recommend_name
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Genting, Matthew","Butters, Gerald"
9,"Hunt, John","Purview, Millicent"


In [41]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("country_club.db")


query = """
SELECT DISTINCT f.name AS facility_name
FROM Facilities f
INNER JOIN Bookings b_member ON f.facid = b_member.facid
    AND b_member.memid <> 0      -- only member bookings
LEFT JOIN Bookings b_guest ON f.facid = b_guest.facid
    AND b_guest.memid = 0       -- guest bookings
WHERE b_guest.facid IS NULL;   
"""

# Load into pandas DataFrame
df_not_guest = pd.read_sql_query(query, conn)

# Display the result
df_not_guest

,facility_name


In [42]:
import sqlite3
import pandas as pd

# Connect to SQLite database
conn = sqlite3.connect("country_club.db")


query = """
SELECT f.name AS facility_name, strftime('%Y-%m',b.starttime) as book_month, count(*)
FROM Facilities f
INNER JOIN Bookings b ON f.facid = b.facid
WHERE b.memid <> 0
GROUP BY  f.name, strftime('%Y-%m',b.starttime) ;   
"""

# Load into pandas DataFrame
df_month = pd.read_sql_query(query, conn)

# Display the result
df_month

,facility_name,book_month,count(*)
0,Badminton Court,2012-07,51
1,Badminton Court,2012-08,132
2,Badminton Court,2012-09,161
3,Massage Room 1,2012-07,77
4,Massage Room 1,2012-08,153
5,Massage Room 1,2012-09,191
6,Massage Room 2,2012-07,4
7,Massage Room 2,2012-08,9
8,Massage Room 2,2012-09,14
9,Pool Table,2012-07,103
